### Setup

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.append(os.path.dirname(os.getcwd()))

from IPython.display import Image, display

import constants, utils
from multi_agent_graph import get_multi_agent_summarizer_graph
from data_classes.book_summarizer_state import BookSummarizerState, EPUB

### Execute the Graph


In [2]:
config = {"recursion_limit": 50}
inputs = {
    "book_name": constants.BOOK_NAME, 
    "book_path": constants.BOOK_PATH, 
    "num_chapters_to_summarize": constants.NUM_CHAPTERS_TO_SUMMARIZE, 
    "verbose": constants.VERBOSE,
    "summaries_dir": constants.BOOK_SUMMARIES_DIR,
}

graph = get_multi_agent_summarizer_graph()
app = graph.compile()
# display(Image(app.get_graph(xray=True).draw_mermaid_png())) 

for event in app.stream(inputs, config=config):
    print("Event:", event)


State:  <data_classes.book_summarizer_state.BookSummarizerState object at 0x7fdaf1156230> 
 SAVING SUMMARY


 33 chapters found in the book
Event: {'Epub Loader': {'book_name': 'eat_that_frog', 'verbose': 1, 'book_path': '/home/pedro/Documents/GitHub/SwarmGPT/data/books/eat_that_frog.epub', 'book': <book_summarizer.data_classes.epub.EPUB object at 0x7fdaf1157df0>, 'chapter_titles_all': ['Cover Page', 'Title Page', 'Copyright Page', 'Dedication', 'Contents', 'Preface', 'Introduction: Eat That Frog', '1 Set the Table', '2 Plan Every Day in Advance', '3 Apply the 80/20 Rule to Everything', '4 Consider the Consequences', '5 Practice Creative Procrastination', '6 Use the ABCDE Method Continually', '7 Focus on Key Result Areas', '8 Apply the Law of Three', '9 Prepare Thoroughly Before You Begin', '10 Take It One Oil Barrel at a Time', '11 Upgrade Your Key Skills', '12 Identify Your Key Constraints', '13 Put the Pressure on Yourself', '14 Motivate Yourself into Action', '15 Technology Is a Te

### BooookScore

In [3]:
from BooookScore.booookscore.score import Scorer 
import json

# Define variables
book_name = constants.BOOK_NAME
model = constants.MODEL_NAME

# Configuration variables
api = "openai"
api_key_path = "BooookScore/api.txt"
chunk_size = 4096
max_context_len = 8192

# Create results directory structure
annotations_output_path = constants.BOOOOKSCORE_RESULTS_DIR + "annotations.json"
score_output_path = constants.BOOOOKSCORE_RESULTS_DIR + "score.txt"


# Ensure the directories exist
os.makedirs(constants.BOOOOKSCORE_RESULTS_DIR, exist_ok=True)

print("Scoring the summaries...")
scorer = Scorer(model=model,api=api,api_key=api_key_path,summ_path=constants.FINAL_BOOK_SUMMARY_PATH,annot_path=annotations_output_path,
    template_path="BooookScore/prompts/get_annotations.txt",v2=False,)
score = scorer.get_score()
print(f"BooookScore = {score}")

# Open the summaries json file which contains a dictionary of the summaries 
with open(constants.FINAL_BOOK_SUMMARY_PATH, "r") as f:
    summary_dict = json.load(f)
n_words = len(summary_dict[book_name].split())
print(f"Summary Length (Words): {n_words}")

# Save the score and summary length to a json file
results = {"BooookScore": score, "n_words": n_words}
with open(score_output_path, "w") as f:
    json.dump(results, f, indent=4)


print(f"All files have been saved to {constants.BOOOOKSCORE_RESULTS_DIR}")

Scoring the summaries...
BooookScore = 0.41666666666666663
Summary Length (Words): 166
All files have been saved to /home/pedro/Documents/GitHub/SwarmGPT/results/booookscore/gpt-3.5-turbo/eat_that_frog/
